# Gaussian Quadrature

## Introduction

Gaussian quadrature is a family of numerical integration methods that achieve optimal accuracy for polynomial integrands. Unlike Newton-Cotes formulas (e.g., trapezoidal rule, Simpson's rule) that use equally spaced nodes, Gaussian quadrature strategically places both the nodes and weights to maximize the degree of polynomial exactness.

## Theoretical Foundation

### The Quadrature Problem

We seek to approximate the integral:

$$\int_a^b w(x) f(x) \, dx \approx \sum_{i=1}^{n} w_i f(x_i)$$

where $w(x)$ is a weight function, $\{x_i\}$ are the quadrature nodes, and $\{w_i\}$ are the corresponding weights.

### Degree of Exactness

A quadrature rule with $n$ nodes has **degree of exactness** $d$ if it integrates all polynomials of degree $\leq d$ exactly. Newton-Cotes rules with $n$ nodes achieve degree $d = n-1$ (or $n$ for closed rules with even $n$). Gaussian quadrature achieves the optimal degree:

$$d = 2n - 1$$

This means $n$ Gaussian nodes can exactly integrate polynomials up to degree $2n-1$.

### Orthogonal Polynomials

The key insight is that Gaussian nodes are the roots of orthogonal polynomials. For the weight function $w(x)$ on interval $[a, b]$, polynomials $\{p_n(x)\}$ are orthogonal if:

$$\int_a^b w(x) p_m(x) p_n(x) \, dx = 0 \quad \text{for } m \neq n$$

### Gauss-Legendre Quadrature

The most common variant uses $w(x) = 1$ on $[-1, 1]$, with nodes being roots of Legendre polynomials $P_n(x)$. Legendre polynomials satisfy:

$$P_n(x) = \frac{1}{2^n n!} \frac{d^n}{dx^n} (x^2 - 1)^n$$

The weights are given by:

$$w_i = \frac{2}{(1 - x_i^2)[P_n'(x_i)]^2}$$

### Change of Variables

To integrate over $[a, b]$ instead of $[-1, 1]$, we use:

$$\int_a^b f(x) \, dx = \frac{b-a}{2} \int_{-1}^{1} f\left(\frac{b-a}{2}t + \frac{a+b}{2}\right) dt$$

### Error Analysis

For a function $f \in C^{2n}[a, b]$, the error of $n$-point Gauss-Legendre quadrature is:

$$E_n[f] = \frac{(b-a)^{2n+1} (n!)^4}{(2n+1)[(2n)!]^3} f^{(2n)}(\xi)$$

for some $\xi \in (a, b)$. This shows spectral convergence for smooth functions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import roots_legendre, erf
from scipy import integrate

# Set up matplotlib for publication-quality figures
plt.rcParams.update({
    'font.size': 11,
    'axes.labelsize': 12,
    'axes.titlesize': 13,
    'legend.fontsize': 10,
    'figure.figsize': (10, 8)
})

## Implementation

### Gauss-Legendre Quadrature Function

We implement a general Gauss-Legendre quadrature routine that can integrate over arbitrary intervals $[a, b]$.

In [ ]:
def gauss_legendre_quadrature(f, a, b, n):
    """
    Compute the integral of f from a to b using n-point Gauss-Legendre quadrature.
    
    Parameters
    ----------
    f : callable
        Function to integrate
    a, b : float
        Integration limits
    n : int
        Number of quadrature points
    
    Returns
    -------
    float
        Approximation of the integral
    """
    # Get nodes and weights on [-1, 1]
    nodes, weights = roots_legendre(n)
    
    # Transform to [a, b]
    transformed_nodes = 0.5 * (b - a) * nodes + 0.5 * (a + b)
    
    # Apply quadrature formula with Jacobian
    integral = 0.5 * (b - a) * np.sum(weights * f(transformed_nodes))
    
    return integral

### Comparison Quadrature Methods

For comparison, we implement the trapezoidal rule and Simpson's rule.

In [ ]:
def trapezoidal_rule(f, a, b, n):
    """
    Composite trapezoidal rule with n subintervals.
    """
    x = np.linspace(a, b, n + 1)
    y = f(x)
    h = (b - a) / n
    return h * (0.5 * y[0] + np.sum(y[1:-1]) + 0.5 * y[-1])

def simpsons_rule(f, a, b, n):
    """
    Composite Simpson's rule with n subintervals (n must be even).
    """
    if n % 2 != 0:
        n += 1
    x = np.linspace(a, b, n + 1)
    y = f(x)
    h = (b - a) / n
    return h / 3 * (y[0] + 4 * np.sum(y[1:-1:2]) + 2 * np.sum(y[2:-1:2]) + y[-1])

## Test Functions

We test the quadrature methods on several functions with known analytical integrals:

1. **Polynomial**: $f(x) = x^4 - 3x^2 + 1$, $\int_0^2 f(x)\,dx = \frac{26}{15}$

2. **Oscillatory**: $f(x) = \sin(x)$, $\int_0^\pi f(x)\,dx = 2$

3. **Smooth non-polynomial**: $f(x) = e^{-x^2}$, $\int_0^1 f(x)\,dx = \frac{\sqrt{\pi}}{2}\text{erf}(1)$

4. **Runge function**: $f(x) = \frac{1}{1+25x^2}$, $\int_{-1}^{1} f(x)\,dx = \frac{2}{5}\arctan(5)$

In [ ]:
# Define test functions and their exact integrals
test_functions = [
    {
        'name': 'Polynomial: $x^4 - 3x^2 + 1$',
        'f': lambda x: x**4 - 3*x**2 + 1,
        'a': 0, 'b': 2,
        'exact': 26/15
    },
    {
        'name': 'Oscillatory: $\\sin(x)$',
        'f': lambda x: np.sin(x),
        'a': 0, 'b': np.pi,
        'exact': 2.0
    },
    {
        'name': 'Gaussian: $e^{-x^2}$',
        'f': lambda x: np.exp(-x**2),
        'a': 0, 'b': 1,
        'exact': 0.5 * np.sqrt(np.pi) * erf(1)
    },
    {
        'name': 'Runge: $1/(1+25x^2)$',
        'f': lambda x: 1 / (1 + 25*x**2),
        'a': -1, 'b': 1,
        'exact': 2/5 * np.arctan(5)
    }
]

# Test with varying number of points
n_values = np.arange(2, 21)

print("Convergence Analysis of Quadrature Methods")
print("=" * 60)

In [ ]:
# Compute errors for all methods and functions
results = {}

for test in test_functions:
    f = test['f']
    a, b = test['a'], test['b']
    exact = test['exact']
    
    errors_gauss = []
    errors_trap = []
    errors_simp = []
    
    for n in n_values:
        # Gaussian quadrature
        gauss_result = gauss_legendre_quadrature(f, a, b, n)
        errors_gauss.append(abs(gauss_result - exact))
        
        # Trapezoidal (use same number of function evaluations)
        trap_result = trapezoidal_rule(f, a, b, n)
        errors_trap.append(abs(trap_result - exact))
        
        # Simpson's
        simp_result = simpsons_rule(f, a, b, n if n % 2 == 0 else n + 1)
        errors_simp.append(abs(simp_result - exact))
    
    results[test['name']] = {
        'gauss': np.array(errors_gauss),
        'trapezoid': np.array(errors_trap),
        'simpson': np.array(errors_simp)
    }

# Print summary for one function
print("\nExample: Runge function 1/(1+25x²)")
print("-" * 50)
print(f"{'n':>3} {'Gauss Error':>15} {'Trap Error':>15} {'Simp Error':>15}")
print("-" * 50)
for i, n in enumerate(n_values[::3]):  # Every 3rd value
    idx = i * 3
    ge = results['Runge: $1/(1+25x^2)$']['gauss'][idx]
    te = results['Runge: $1/(1+25x^2)$']['trapezoid'][idx]
    se = results['Runge: $1/(1+25x^2)$']['simpson'][idx]
    print(f"{n:>3} {ge:>15.2e} {te:>15.2e} {se:>15.2e}")

## Visualization

We create a comprehensive visualization showing:
1. Convergence of quadrature errors for different test functions
2. Comparison of Gauss-Legendre nodes versus uniform spacing
3. Weight distribution for different orders

In [ ]:
fig = plt.figure(figsize=(14, 10))

# Plot 1: Convergence comparison for all test functions
ax1 = fig.add_subplot(2, 2, 1)
colors = plt.cm.tab10(np.linspace(0, 1, 4))
for idx, (name, data) in enumerate(results.items()):
    # Avoid log(0) by replacing zeros with machine epsilon
    gauss_err = np.maximum(data['gauss'], 1e-16)
    ax1.semilogy(n_values, gauss_err, 'o-', color=colors[idx], 
                 label=name.split(':')[1].strip(), markersize=4)

ax1.set_xlabel('Number of Quadrature Points (n)')
ax1.set_ylabel('Absolute Error')
ax1.set_title('Gauss-Legendre Convergence')
ax1.legend(loc='upper right', fontsize=8)
ax1.grid(True, alpha=0.3)
ax1.set_ylim([1e-16, 1e1])

# Plot 2: Method comparison for Runge function
ax2 = fig.add_subplot(2, 2, 2)
runge_data = results['Runge: $1/(1+25x^2)$']
ax2.semilogy(n_values, np.maximum(runge_data['gauss'], 1e-16), 'bo-', 
             label='Gauss-Legendre', markersize=5)
ax2.semilogy(n_values, np.maximum(runge_data['trapezoid'], 1e-16), 'rs-', 
             label='Trapezoidal', markersize=5)
ax2.semilogy(n_values, np.maximum(runge_data['simpson'], 1e-16), 'g^-', 
             label="Simpson's", markersize=5)

ax2.set_xlabel('Number of Points (n)')
ax2.set_ylabel('Absolute Error')
ax2.set_title('Method Comparison: Runge Function')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim([1e-16, 1e0])

# Plot 3: Node distribution comparison
ax3 = fig.add_subplot(2, 2, 3)
for n in [5, 10, 15, 20]:
    nodes, _ = roots_legendre(n)
    ax3.scatter(nodes, [n]*n, c='blue', s=30, alpha=0.6)
    # Uniform nodes for comparison
    uniform = np.linspace(-1, 1, n)
    ax3.scatter(uniform, [n + 0.3]*n, c='red', s=20, alpha=0.4, marker='x')

ax3.set_xlabel('Node Position')
ax3.set_ylabel('Number of Points (n)')
ax3.set_title('Gauss-Legendre (blue) vs Uniform (red) Nodes')
ax3.set_yticks([5, 10, 15, 20])
ax3.grid(True, alpha=0.3)

# Plot 4: Weights for different n
ax4 = fig.add_subplot(2, 2, 4)
for n in [3, 5, 7, 10]:
    nodes, weights = roots_legendre(n)
    ax4.plot(nodes, weights, 'o-', label=f'n={n}', markersize=6)

ax4.set_xlabel('Node Position')
ax4.set_ylabel('Weight')
ax4.set_title('Gauss-Legendre Weights')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved to plot.png")

## Analysis of Results

### Convergence Behavior

The results demonstrate several key properties of Gaussian quadrature:

1. **Polynomial exactness**: For the polynomial test function $x^4 - 3x^2 + 1$ (degree 4), Gauss-Legendre with $n \geq 3$ points achieves machine precision since $2(3) - 1 = 5 \geq 4$.

2. **Spectral convergence**: For smooth functions, the error decreases exponentially with $n$. This is evident in the Gaussian $e^{-x^2}$ and oscillatory $\sin(x)$ cases.

3. **Superior accuracy**: Gaussian quadrature consistently outperforms trapezoidal and Simpson's rules by several orders of magnitude for the same number of function evaluations.

### Node Distribution

Gauss-Legendre nodes cluster near the endpoints of $[-1, 1]$. This distribution:
- Counteracts the Runge phenomenon that plagues polynomial interpolation with uniform nodes
- Optimally samples the function to maximize integration accuracy
- Resembles the Chebyshev distribution $x_k = \cos\left(\frac{2k-1}{2n}\pi\right)$

### Weight Symmetry

The weights exhibit symmetry about $x = 0$ due to the symmetry of Legendre polynomials. Interior nodes receive larger weights than boundary nodes, opposite to the behavior in Newton-Cotes formulas.

## Advanced Application: Adaptive Quadrature

We implement a simple adaptive scheme that subdivides intervals where error estimates are large.

In [ ]:
def adaptive_gauss(f, a, b, tol=1e-10, n_low=5, n_high=10, max_depth=50):
    """
    Adaptive Gauss-Legendre quadrature.
    
    Uses the difference between n_low and n_high point rules as error estimate.
    """
    def quad_adaptive(f, a, b, depth):
        if depth > max_depth:
            return gauss_legendre_quadrature(f, a, b, n_high)
        
        I_low = gauss_legendre_quadrature(f, a, b, n_low)
        I_high = gauss_legendre_quadrature(f, a, b, n_high)
        
        error = abs(I_high - I_low)
        
        if error < tol:
            return I_high
        else:
            mid = (a + b) / 2
            left = quad_adaptive(f, a, mid, depth + 1)
            right = quad_adaptive(f, mid, b, depth + 1)
            return left + right
    
    return quad_adaptive(f, a, b, 0)

# Test on a challenging function with a sharp peak
f_peak = lambda x: 1 / (1 + 1000 * (x - 0.5)**2)
exact_peak = np.arctan(np.sqrt(1000) * 0.5) / np.sqrt(1000) + np.arctan(np.sqrt(1000) * 0.5) / np.sqrt(1000)

# Compute with scipy for reference
exact_peak, _ = integrate.quad(f_peak, 0, 1)

# Test methods
adaptive_result = adaptive_gauss(f_peak, 0, 1, tol=1e-10)
fixed_result = gauss_legendre_quadrature(f_peak, 0, 1, n=50)

print("\nAdaptive Quadrature Test: Sharp Peak Function")
print("="*50)
print(f"Exact (scipy.quad):     {exact_peak:.12f}")
print(f"Adaptive Gauss:         {adaptive_result:.12f}")
print(f"Fixed 50-point Gauss:   {fixed_result:.12f}")
print(f"\nAdaptive error:         {abs(adaptive_result - exact_peak):.2e}")
print(f"Fixed 50-pt error:      {abs(fixed_result - exact_peak):.2e}")

## Other Gaussian Quadrature Variants

Beyond Gauss-Legendre, other variants use different weight functions:

| Name | Weight $w(x)$ | Interval | Orthogonal Polynomials |
|------|---------------|----------|------------------------|
| Gauss-Legendre | $1$ | $[-1, 1]$ | Legendre |
| Gauss-Chebyshev | $(1-x^2)^{-1/2}$ | $[-1, 1]$ | Chebyshev |
| Gauss-Laguerre | $e^{-x}$ | $[0, \infty)$ | Laguerre |
| Gauss-Hermite | $e^{-x^2}$ | $(-\infty, \infty)$ | Hermite |
| Gauss-Jacobi | $(1-x)^\alpha(1+x)^\beta$ | $[-1, 1]$ | Jacobi |

These are essential in applications with natural weight functions, such as quantum mechanics (Hermite for harmonic oscillator) and probability (Hermite for Gaussian distributions).

## Conclusions

Gaussian quadrature represents the optimal strategy for numerical integration when function evaluations are expensive. Key takeaways:

1. **Optimal efficiency**: $n$ points yield degree of exactness $2n-1$, twice that of Newton-Cotes methods

2. **Spectral convergence**: Errors decrease exponentially for smooth functions

3. **Strategic node placement**: Non-uniform nodes avoid Runge-type oscillations

4. **Theoretical elegance**: Deep connections to orthogonal polynomials and approximation theory

5. **Practical robustness**: Widely used in finite element methods, computational physics, and statistical computing

For integrands with singularities or discontinuities, adaptive methods or specialized quadrature rules (e.g., Gauss-Kronrod for error estimation) should be employed.